# Approche supervisée

Nous allons tester et comparer différents algorithmes de classification supervisée afin d'identifier les tags pour chaque texte.

## Chargement des données

On vient importer les données formatées. Il y a deux fichiers pour chaque approche de vectorisation : un pour le jeu d'entraînement, l'autre pour le jeu d'évaluation. Nous allons charger tous ces jeux de données dans un dictionnaire, 

In [1]:
#import des bibliothèques requises
import pandas as pd
import numpy as np
import re

In [2]:
#On liste nos 5 vectorisation utilisées
liste_vector = ['bow', 'tf', 'w2v', 'use', 'bert']
#On remplit le dictionnaire de dataframes
dict_data = {}
for vect in liste_vector:
    dict_data[f"train_{vect}"] = pd.read_csv(f'database_{vect}.csv').dropna()
    dict_data[f"test_{vect}"] = pd.read_csv(f'database_{vect}_test.csv').dropna()
#On affiche les tailles des dataframes obtenus
for key, item in dict_data.items():
    print(key)
    print(item.shape)

train_bow
(21703, 1003)
test_bow
(5425, 1003)
train_tf
(21703, 1003)
test_tf
(5425, 1003)
train_w2v
(21703, 103)
test_w2v
(5425, 103)
train_use
(21703, 515)
test_use
(5425, 515)
train_bert
(21645, 771)
test_bert
(5404, 771)


## Evaluation des performances

Il nous faudra évaluer l'efficacité de nos algorithmes. En plus du temps d'éxécution de chaque algorithme, nous allons calculer la précision (**accuracy score**) de la classification et le **score F1**, pour le jeu d'entraînement et le jeu de test.

In [3]:
from sklearn import metrics

def indic_performances(modele, X_train, y_train, X_test, y_test):
    y_simul = modele.predict(X_train)
    y_pred = modele.predict(X_test)    
    #Nous utiliserons comme indicateurs l'accuracy et le F1 score sur les jeux de train et de test
    accuracy_train = metrics.accuracy_score(y_train, y_simul)
    accuracy_test = metrics.accuracy_score(y_test, y_pred)
    f1_train = metrics.f1_score(y_train, y_simul, average='micro')
    f1_test = metrics.f1_score(y_test, y_pred, average='micro')
    return (accuracy_train, accuracy_test, f1_train, f1_test)

## Comparaison des algorithmes

On va utiliser plusieurs algorithmes de classification avec les hyperparamètres par défaut sur chacune de nos vectorisations, avant d'évaluer leurs performances pour les comparer. On pourra ensuite optimiser le couple vectorisation-algorithme le plus efficace. On utilisera la *Régression logistique* comme baseline, une *Régression Ridge*, un *Random Forest*, un *XGBoost* et un *Multi Layer Perceptron*. On mettra en place l'*early stopping* quand c'est possible, afin que nos algorithmes cessent de tourner s'ils ne progressent pas dans l'apprentissage.

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
import timeit

#On définit une fonction mettant en place une classification multilabel Random Forest
#et évaluant ses performances sur les jeux de train et de test
def classif_random_forest(X_train, y_train, X_test, y_test):
    forest = RandomForestClassifier(random_state=1)
    classif = MultiOutputClassifier(forest, n_jobs=-1)
    #On retient le temps de démarrage du calcul, avant de calquer le modèle sur le jeu de train
    start_time = timeit.default_timer()
    classif.fit(X_train, y_train)
    #On récupère le temps de calcul, ainsi que les prédictions sur les deux jeux pour évaluer les performances
    elapsed = timeit.default_timer() - start_time
    #On récupère les indicateurs de performances de ce modèle
    scores = indic_performances(classif, X_train, y_train, X_test, y_test)
    return (scores, elapsed)

#On définit une fonction mettant en place une classification multilabel XGBoost
#et évaluant ses performances sur les jeux de train et de test
def classif_XGBoost(X_train, y_train, X_test, y_test):
    xgbc = XGBClassifier(seed=42)
    classif = MultiOutputClassifier(xgbc, n_jobs=-1)
    #On retient le temps de démarrage du calcul, avant de calquer le modèle sur le jeu de train
    start_time = timeit.default_timer()
    classif.fit(X_train, y_train)
    #On récupère le temps de calcul, ainsi que les prédictions sur les deux jeux pour évaluer les performances
    elapsed = timeit.default_timer() - start_time
    #On récupère les indicateurs de performances de ce modèle
    scores = indic_performances(classif, X_train, y_train, X_test, y_test)
    return (scores, elapsed)

#On définit une fonction mettant en place une classification multilabel Régression logistique
#et évaluant ses performances sur les jeux de train et de test
def classif_logistic_regression(X_train, y_train, X_test, y_test):
    logreg = LogisticRegression(multi_class='multinomial')
    classif = MultiOutputClassifier(logreg, n_jobs=-1)
    #On retient le temps de démarrage du calcul, avant de calquer le modèle sur le jeu de train
    start_time = timeit.default_timer()
    classif.fit(X_train, y_train)
    #On récupère le temps de calcul, ainsi que les prédictions sur les deux jeux pour évaluer les performances
    elapsed = timeit.default_timer() - start_time
    #On récupère les indicateurs de performances de ce modèle
    scores = indic_performances(classif, X_train, y_train, X_test, y_test)
    return (scores, elapsed)


#On définit une fonction mettant en place une classification multilabel MLP
#et évaluant ses performances sur les jeux de train et de test
def classif_MLP(X_train, y_train, X_test, y_test):
    mlp = MLPClassifier(random_state=42)
    classif = MultiOutputClassifier(mlp, n_jobs=-1)
    #On retient le temps de démarrage du calcul, avant de calquer le modèle sur le jeu de train
    start_time = timeit.default_timer()
    classif.fit(X_train, y_train)
    #On récupère le temps de calcul, ainsi que les prédictions sur les deux jeux pour évaluer les performances
    elapsed = timeit.default_timer() - start_time
    #On récupère les indicateurs de performances de ce modèle
    scores = indic_performances(classif, X_train, y_train, X_test, y_test)
    return (scores, elapsed)

#On définit une fonction mettant en place une classification multilabel Régression Ridge
#et évaluant ses performances sur les jeux de train et de test
def classif_ridge(X_train, y_train, X_test, y_test, alpha=1.0):
    ridge = RidgeClassifier(alpha=alpha)
    classif = MultiOutputClassifier(ridge, n_jobs=-1)
    #On retient le temps de démarrage du calcul, avant de calquer le modèle sur le jeu de train
    start_time = timeit.default_timer()
    classif.fit(X_train, y_train)
    #On récupère le temps de calcul, ainsi que les prédictions sur les deux jeux pour évaluer les performances
    elapsed = timeit.default_timer() - start_time
    #On récupère les indicateurs de performances de ce modèle
    scores = indic_performances(classif, X_train, y_train, X_test, y_test)
    return (scores, elapsed)

#On va lister les fonctions implémentées dans un dictionnaire
dict_classifiers = {'Régression logistique' : classif_logistic_regression,
                    'Ridge regression' : classif_ridge,
                    'Random Forest classifier' : classif_random_forest,
                    'XGBoost Classifier' : classif_XGBoost,
                    'Multi-layer Perceptron' : classif_MLP}

D:\Anaconda\envs\IML\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


La classification que l'on souhaite faire est une classification dite **multilabel** : chaque entrée peut appartenir à un nombre variable de classes simultanément. Il nous faut donc formater l'étiquette comme une liste de variables binaires pour chaque tag, tout en précisant aux algorithmes que ces classes peuvent se chevaucher. La seule exception est le *Ridge Regression Classifier* qui ne supporte pas le multilabel, et utilise une approche *OveVsRest* pour sélectionner **le** tag le plus pertinent.

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

#Cette fonction permet de transformer le string contenant les tags entre chevrons en une liste de tags individuels
def formatage_tags(tags):
    temp = re.findall(r'<[^>]+>', tags)
    return [mot[1:-1] for mot in temp]

#Cette fonction nous servira à afficher les scores de performances obtenus par chaque algorithme
def affichage(results, N_train, N_test):
    print(f"Accuracy train ({N_train} inputs) : {results[0][0]}\n Accuracy test ({N_test} inputs) : {results[0][1]}")
    print(f"F1 score train ({N_train} inputs) : {results[0][2]}\n F1 score test ({N_test} inputs) : {results[0][3]}")
    print(f"Elapsed time ({N_train} inputs) : {results[1]}")

On peut désormais effectuer la comparaison proprement dite, en essayant chaque algorithme sur chaque jeu d'entraînement avant d'évaluer ses performances sur le jeu d'entraînement utilisé et le jeu d'évaluation apparié.

In [6]:
#Pour chacune des 5 vectorisations utilisées...
for vect in liste_vector:
    print(f"\n \n \n Vectorization {vect} : \n")
    #On prépare les données à classifier, pour le train et le test
    X_train = dict_data[f"train_{vect}"].drop(columns = ['Tags', 'Texte', 'Texte_clean'])
    X_test = dict_data[f"test_{vect}"].drop(columns = ['Tags', 'Texte', 'Texte_clean'])
    N_inputs = (X_train.shape[0], X_test.shape[0])
    #On prépare les étiquettes, pour le train et le test
    mlb = MultiLabelBinarizer()
    y_train = mlb.fit_transform(dict_data[f"train_{vect}"]['Tags'].apply(formatage_tags))
    y_test = mlb.transform(dict_data[f"test_{vect}"]['Tags'].apply(formatage_tags))
    #Pour chacun des classifieurs listés...
    for key, item in dict_classifiers.items():
        #On entraîne le classifieur sur les données, en récupérant les performances
        results = item(X_train, y_train, X_test, y_test)
        #On affiche le classifieur utilisé et les indicateurs de performances
        print(key)
        affichage(results, N_inputs[0], N_inputs[1])
        print('\n')


 
 
 Vectorization bow : 

Régression logistique
Accuracy train (21703 inputs) : 0.36985670183845554
 Accuracy test (5425 inputs) : 0.14930875576036867
F1 score train (21703 inputs) : 0.7001909966680143
 F1 score test (5425 inputs) : 0.4896544133832269
Elapsed time (21703 inputs) : 180.2509641


Ridge regression
Accuracy train (21703 inputs) : 0.11740312399207482
 Accuracy test (5425 inputs) : 0.10193548387096774
F1 score train (21703 inputs) : 0.38311929198537337
 F1 score test (5425 inputs) : 0.3482085846044697
Elapsed time (21703 inputs) : 38.094155400000005


Random Forest classifier
Accuracy train (21703 inputs) : 0.9972814818227895
 Accuracy test (5425 inputs) : 0.14967741935483872
F1 score train (21703 inputs) : 0.9989731756561868
 F1 score test (5425 inputs) : 0.4388027792624265
Elapsed time (21703 inputs) : 404.1302815


XGBoost Classifier
Accuracy train (21703 inputs) : 0.5132930931207668
 Accuracy test (5425 inputs) : 0.18912442396313364
F1 score train (21703 inputs) : 0.79

Les deux associations les plus intéressantes en termes de résultats seraient la vectorisation TF-IDF avec le classifieur Random Forest, qui atteint $15,6\%$ de précision sur le jeu de test, ainsi que la vectorisation USE avec le classifieur MLP qui atteint $22,9\%$ de précision sur le jeu de test. C'est ce deuxième couple (USE et MLP) que nous retiendrons comme modèle.

# Optimisation des paramètres

On va faire une validation croisée pour sélectionner les paramètres les plus pertinents pour notre classifieur.

In [6]:
#On charge à nouveau les databases
data_train = pd.read_csv('database_use.csv').dropna()
data_test = pd.read_csv('database_use_test.csv').dropna()
#On vérifie la taille des jeux de données
print("Jeu de test :")
print(data_test.shape)
print("Jeu de train :")
print(data_train.shape)
data_train.head()

Jeu de test :
(5425, 515)
Jeu de train :
(21703, 515)


,Tags,Texte,Texte_clean,0,1,2,3,4,5,6,...,502,503,504,505,506,507,508,509,510,511
0,<javascript>,Mercator longitude and latitude calculations t...,mercator longitude latitude calculation map uk...,-0.028300,-0.066435,-0.055056,0.021637,0.065491,0.041183,-0.059594,...,0.015944,-0.067285,0.037467,-0.030200,0.055697,0.065050,-0.007969,0.033120,-0.005994,-0.021647
1,<c++>,Starting point for learning CAD/CAE file forma...,point cad cae file stress analysis software un...,-0.061839,-0.068042,0.013900,0.061812,0.033550,-0.024985,-0.059817,...,0.025068,-0.065279,0.012942,0.016776,-0.069897,0.066179,0.017820,0.060028,-0.029135,-0.063384
2,<c++><performance><optimization>,How to overload std::swap() is used by many st...,overload stdswap container assignment implemen...,0.014252,-0.070463,0.043723,-0.002921,0.039176,0.047006,0.065745,...,-0.017394,-0.072381,0.029358,-0.070786,0.019498,-0.017685,0.020693,0.073793,-0.004406,0.004228
3,<c#><.net>,Returning DataTables in WCF/.NET I have a WCF ...,datatables wcf wcf service return know topic d...,-0.054523,-0.065295,0.062613,-0.017571,0.003668,0.039311,0.055374,...,0.001709,-0.068075,-0.000101,-0.020418,0.046734,-0.062165,0.043847,0.068074,-0.017898,-0.037102
4,<c#><.net><performance><datetime>,Is DateTime.Now the best way to measure a func...,way measure function performance bottleneck me...,-0.018517,-0.052889,-0.037226,-0.042735,0.037281,-0.037934,-0.057746,...,0.046747,-0.038951,0.022992,-0.069126,0.063291,-0.013801,0.073607,0.076682,-0.030643,0.031145


In [7]:
#On isole les variables et les étiquettes des deux jeux de données
X_train = data_train.drop(columns = ['Tags', 'Texte', 'Texte_clean'])
X_test = data_test.drop(columns = ['Tags', 'Texte', 'Texte_clean'])
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(data_train['Tags'].apply(formatage_tags))
y_test = mlb.transform(data_test['Tags'].apply(formatage_tags))

In [10]:
from sklearn.model_selection import GridSearchCV

#On place dans un dictionnaire les paramètres que nous allons utiliser et éventuellement modifier dans notre VC
parameters = {
    'activation' : ['logistic', 'relu'],
    'alpha' : np.logspace(-4,-2,3),
    'learning_rate' : ['constant', 'adaptive'],
    'random_state' : [42],
    'early_stopping' : [True],
    'validation_fraction' : [0.1],
    'beta_1' : [0.5, 0.9, 0.99],
    'beta_2' : [0.9, 0.99, 0.999],
}

In [12]:
parameters = {
    'activation' : ['logistic', 'relu'],
    'alpha' : np.logspace(-4,-2,3),
    'learning_rate' : ['constant', 'adaptive'],
    'random_state' : [42],
    'early_stopping' : [True],
    'validation_fraction' : [0.1],
}

In [13]:
#On initialise la validation croisée
cross_val = GridSearchCV(MLPClassifier(), parameters, scoring=metrics.accuracy_score, n_jobs=-1)
#On l'applique sur nos données
cross_val.fit(X_train, y_train)
#On récupère les paramètres les plus pertinents
print(cross_val.best_params_)

D:\Anaconda\envs\IML\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid

D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

{'activation': 'logistic', 'alpha': 0.0001, 'early_stopping': True, 'learning_rate': 'constant', 'random_state': 42, 'validation_fraction': 0.1}


In [16]:
parameters = {
    'activation' : ['logistic'],
    'alpha' : [0.0001],
    'learning_rate' : ['constant'],
    'random_state' : [42],
    'early_stopping' : [True],
    'validation_fraction' : [0.1],
    'beta_1' : [0.5, 0.9, 0.99],
    'beta_2' : [0.9, 0.99, 0.999],
}

In [17]:
#On initialise la validation croisée
cross_val = GridSearchCV(MLPClassifier(), parameters, scoring=metrics.accuracy_score, n_jobs=-1)
#On l'applique sur nos données
cross_val.fit(X_train, y_train)
#On récupère les paramètres les plus pertinents
print(cross_val.best_params_)

D:\Anaconda\envs\IML\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan]
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature nam

D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


{'activation': 'logistic', 'alpha': 0.0001, 'beta_1': 0.5, 'beta_2': 0.9, 'early_stopping': True, 'learning_rate': 'constant', 'random_state': 42, 'validation_fraction': 0.1}


D:\Anaconda\envs\IML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [18]:
#On construit notre classifieur avec les paramètres précédemment identifiés
mlp = MLPClassifier(**cross_val.best_params_)
mlp_classif = MultiOutputClassifier(mlp, n_jobs=-1)
#On l'entraîne une dernière fois
mlp_classif.fit(X_train, y_train)
#On récupère et affiche les scores de performances de notre algorithme
scores = indic_performances(mlp_classif, X_train, y_train, X_test, y_test)
print("Performances finales : \n")
print(f"Accuracy score train  : {scores[0]}\n Accuracy score test : {scores[1]}\n")
print(f"F1 score train  : {scores[2]}\n F1 score test : {scores[3]}\n")

Performances finales : 

Accuracy score train  : 0.25217711837073215
 Accuracy score test : 0.21751152073732719

F1 score train  : 0.6239877769289535
 F1 score test : 0.5877824267782428



Il va nous falloir exporter le modèle une fois qu'il est entraîné. Nous utiliserons pour cela la librairie pickle. N'oublions pas d'exporter le *multilabel binarizer* pour pouvoir transformer les étiquettes renvoyées par le classifieur en tags compréhensibles.

In [18]:
import pickle

#On exporte le classifieur
with open('tag_detection_classifier', 'wb') as f1:
    pickle.dump(mlp_classif, f1)
#On exporte le binarizer
with open('tag_detection_binarizer', 'wb') as f2:
    pickle.dump(mlb, f2)    